In [93]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

In [20]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b

In [94]:
API_URL = 'http://10.92.55.4:5000'

stuID = 28374 #Enter Your ID

#Server's Identitiy public key
IKey_Ser = Point(0xce1a69ecc226f9e667856ce37a44e50dbea3d58e3558078baee8fe5e017a556d, 0x13ddaf97158206b1d80258d7f6a6880e7aaf13180e060bb1e94174e419a4a093, E)
# Use the values in the project description document to form the server's IK as a point on the EC. Note that the values should be in decimal.


In [95]:
def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())

def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    else:
        print(response.json())
        f = open('Identity_Key.txt', 'w')
        f.write("IK.Prv: "+str(IKey_Pr)+"\n"+"IK.Pub.x: "+str(IKey_Pub.x)+"\n"+"IK.Pub.y: "+str(IKey_Pub.y))
        f.close()

def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True


def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    if((response.ok) == False): print(response.json())



In [117]:
def GenerateKey():
  pri = random.randint(0,n-1)
  pub = pri * P
  return pri, pub

In [22]:
IKey_Pr = int.from_bytes(b"this is my secret", byteorder="big") % n
IKey_Pub = IKey_Pr * P

In [43]:
def SignMessage(m):
  k = random.randint(1, n-2)

  R = k * P
  r = R.x % n

  r_bytes = r.to_bytes(math.ceil(r.bit_length()/8), 'big')
  if isinstance(m, int):
    m = m.to_bytes(math.ceil(m.bit_length()/8), 'big')
  
  h = SHA3_256.new(r_bytes + m)
  h = h.digest()
  h = int.from_bytes(h,"big")
  h = h % n
  
  s = (k + IKey_Pr * h) % n
  
  return h, s

In [83]:
def VerifySignature(m, h, s, Key_Pub):
  V = s * P - h * Key_Pub
  v = V.x % n

  v_bytes = v.to_bytes(math.ceil(v.bit_length()/8), 'big')
  if isinstance(m, int):
    m = m.to_bytes(math.ceil(m.bit_length()/8), 'big')

  h2 = SHA3_256.new(v_bytes + m)
  h2 = h2.digest()
  h2 = int.from_bytes(h2,"big") % n
  h2 = h2 % n
  return h2 == h

In [73]:
h, s = SignMessage(stuID)

In [77]:
VerifySignature(stuID, h, s, IKey_Pub)

True

In [27]:
IKRegReq(h, s, IKey_Pub.x, IKey_Pub.y)

Sending message is:  {'ID': 28374, 'H': 91524442026738960398099114529467009562870631138826016684572945633488846875208, 'S': 52487043771225581505444596344839468615657118325051274984779581026310898394349, 'IKPUB.X': 14407597495457330413355653872827102679455709776888388952403808005076122499711, 'IKPUB.Y': 99025964784026725443948688290160998982011637383044340806584066476475475584575}


In [29]:
CODE = 462614

In [30]:
IKRegVerify(CODE)

Sending message is:  {'ID': 28374, 'CODE': 462614}
Registered successfully


In [31]:
RCODE = 740117

In [58]:
SPK_A_Pri, SPK_A_Pub = GenerateKey(33232637425696107713632882735238964086585438341980536407357193648214666452195)

In [79]:
SPK_A_message = SPK_A_Pub.x.to_bytes(math.ceil(SPK_A_Pub.x.bit_length()/8), 'big') + SPK_A_Pub.y.to_bytes(math.ceil(SPK_A_Pub.y.bit_length()/8), 'big')
SPK_A_h, SPK_A_s = SignMessage(SPK_A_message)

In [80]:
SPK_S_X, SPK_S_Y, SPK_S_h, SPK_S_s = SPKReg(SPK_A_h, SPK_A_s, SPK_A_Pub.x, SPK_A_Pub.y)
SPK_S_Pub = Point(SPK_S_X, SPK_S_Y, E)

Sending message is:  {'ID': 28374, 'H': 96050651532608626925049414869311378194339351393712219086830868958732537148573, 'S': 44525415036427161336212359121105819214304037234027977106704928780044718131511, 'SPKPUB.X': 35999222419857415020415158017962267267768487712861254018342541626575617043219, 'SPKPUB.Y': 100536607320859216686041236553405992186374232474394814455179841590277269847771}


In [92]:
SPK_S_message = SPK_S_Pub.x.to_bytes(math.ceil(SPK_S_Pub.x.bit_length()/8), 'big') + SPK_S_Pub.y.to_bytes(math.ceil(SPK_S_Pub.y.bit_length()/8), 'big')

VerifySignature(SPK_S_message, SPK_S_h, SPK_S_s, IKey_Ser)

True

In [110]:
# Generate HMAX Key
T = SPK_A_Pri * SPK_S_Pub
U = b'CuriosityIsTheHMACKeyToCreativity' + T.y.to_bytes(math.ceil(T.y.bit_length()/8), 'big') + T.x.to_bytes(math.ceil(T.x.bit_length()/8), 'big')
K_HMAC = SHA3_256.new(U).digest()

In [120]:
# Generate OTKs
OTKs=[]
for i in range(10):
  priv, pub = GenerateKey()
  m = pub.x.to_bytes(math.ceil(pub.x.bit_length()/8), 'big') + pub.y.to_bytes(math.ceil(pub.y.bit_length()/8), 'big')
  hmaci = HMAC.new(K_HMAC, m, digestmod=SHA256).hexdigest()
  OTKReg(i, pub.x, pub.y, hmaci)
  OTKs.append((priv, pub))

Sending message is:  {'ID': 28374, 'KEYID': 0, 'OTKI.X': 27972663979373135832933264867245307338048170062187415390798417213879312620569, 'OTKI.Y': 23568508699570858293469286654702964761790454308867406353293944970610676316172, 'HMACI': '2cce6b8b9d2b423ae976ac7c0dc0349c2a5c549a386b03ba70b2b9ba58968a99'}
OTK with ID number 0 is registered successfully
Sending message is:  {'ID': 28374, 'KEYID': 1, 'OTKI.X': 57187453263583558274552558253625210289996711981430724575619931614910934037506, 'OTKI.Y': 4513124773096275672392905500886905966622685004487935999175470249449727930029, 'HMACI': '718038b54654c8ccf88db4e61c856022f7b92fda680c7970271ab86348f7c0f4'}
OTK with ID number 1 is registered successfully
Sending message is:  {'ID': 28374, 'KEYID': 2, 'OTKI.X': 42909666989721832656052568395214777913805513981560164695865730959251183046737, 'OTKI.Y': 24136968282158388745806642209569965759212858106280024126635484989679457984242, 'HMACI': 'a1d25654d931ea78f30efc330dba8ee99fe331866708d69f806da17719eed295'}

In [121]:
with open("otks.txt", "w+") as f:
  for otk in OTKs:
    f.write(f"{otk[0]} {otk[1].x} {otk[1].y}\n")